In [1]:
import numpy as np
import pandas as pd
from sklearn import neighbors as nn

In [2]:
def read_df(name,i,k,separators):
    df = pd.read_csv(f"{name}{str(i)},{str(10 - i)}.csv",header=None)
    X = []
    Y = []
    for i in range(1,len(separators)):
        X.append(df.iloc[separators[i-1]:separators[i],:-1])
        Y.append(df.iloc[separators[i-1]:separators[i],-1])
    X.append(df.iloc[separators[i]:,:-1])
    Y.append(df.iloc[separators[i]:,-1])
    print(len(X))
    return X,Y





In [3]:
def map_p(i, zero, one, categories_count):
    divider = zero
    if categories_count["class"][i] == 1:
        divider = one
    return int(categories_count["i"][i] * 10000/divider) / 100


In [5]:

zrange = [2]
names = ["P(y)cd/seaG_"]
separators = [0,45000,55000]

for z in zrange:
    for name in names:
        df_cat = []
        X,Y = read_df(name,z,5,separators)
        for x,y in zip(X,Y):
            Knn = nn.KNeighborsClassifier()
            Knn.fit(x,y)
            category = ["safe","borderline","outlier"]
            nearest = Knn.kneighbors(n_neighbors= 5, return_distance= False)
            results = pd.DataFrame(columns = ["i","class", "category"])
            for i,n in enumerate(nearest):
                cl = y[y.index[i]]
                same = 0
                other = 0
                category = "safe"
                for nnn in n:
                    if y[y.index[nnn]] == cl:
                        same += 1
                    else:
                        other += 1
                if same == 0:
                    category = "outlier"
                elif same == 1:
                    category = "rare"
                elif same <= 3:
                    category = "borderline"
                df2 = pd.DataFrame([[i,cl,category]],columns = ["i","class", "category"])
                results = results.append(df2)
            res = results
            zeros = res[res["class"].unique()[0]].count()["i"]
            ones = res[res["class"].unique()[1]].count()["i"]
            if zeros == 0:
                zeros = 1
            if ones == 0:
                ones = 1
            categories_counts = res.groupby(["class","category"]).count()
            categories_counts = categories_counts.reset_index()
            categories_counts["p"] = pd.Series(categories_counts.index).apply(map_p, args=(zeros,ones,categories_counts))
            df_cat.append(categories_counts)
        final = pd.DataFrame(columns = ["class","category","i", "p"])
        for d in df_cat:
            final = pd.concat([final,d],ignore_index=True)
        final.to_csv(f"{name}{str(z)},{str(10 - z)}results.csv")
        print(f"results_{name}{str(z)},{str(10 - z)}results.csv")






3
results_P(y)cd/seaG_2,8results.csv
